<img src="ubiops_logo.svg" width="80">

# Quickstart
Deploy a simple float-to-float deployment in UbiOps using the R client library.


The first step is to [download](https://download-github.ubiops.com/#!/home?url=https://github.com/UbiOps/deployment-template/tree/master/r/deployment_package) a prepared deployment package (the deployment code), and store it in your current working directory.

In [ ]:
deployment_package_zip <- file.path(getwd(), "deployment_package.zip")

if(!file.exists(deployment_package_zip)){
    stop(paste(
        "ERROR: Please, download the prepared deployment package (see link in text above)",
        "and save it in your current working directory"
    ))
}

Add your API token and project name as environment variable.

In [ ]:
Sys.setenv(UBIOPS_PROJECT = "<YOUR_PROJECT_NAME>")
Sys.setenv(UBIOPS_API_TOKEN = "Token <YOUR_API_TOKEN>")

# On-premises installation of UbiOps? Set your API url using:
# Sys.setenv(UBIOPS_API_URL = "https://api.ubiops.com/v2.1")  # replace with your own API url

You can adapt the deployment/pipeline name and version, or leave the default values.

In [ ]:
DEPLOYMENT_NAME <- "simple-tutorial"
DEPLOYMENT_VERSION <- "v1"

In [ ]:
DEPLOYMENT_NAME2 <- "simple-tutorial-2"
DEPLOYMENT_VERSION2 <- "v1"

In [ ]:
PIPELINE_NAME <- "simple-tutorial-pipeline"
PIPELINE_VERSION <- "v1"

## Load R client library
Make sure you installed the UbiOps client library according to the installation instructions.

In [ ]:
library(ubiops)

Initialize the client library, which establishes the connection with UbiOps.

In [ ]:
result <- service_status()
result

## Deploy
Create a deployment.

In [ ]:
deployment <- list(
    name = DEPLOYMENT_NAME,
    description = "A simple deployment that multiplies the input float by a random number.",
    input_type = "structured",
    output_type = "structured",
    input_fields = list(
        list(name="input", data_type="double")
    ),
    output_fields=list(
        list(name="output", data_type="double")
    ),
    labels = list(demo = "r-quickstart")
)
result <- deployments_create(data = deployment)
result

### Create a deployment version

<div class="alert alert-block alert-info">
    
<b>Deployment Mode:</b> The <i>deployment_mode</i> of a deployment version determines what kind of requests will be made to it. For <code>deployment_mode="express"</code>, direct (<i>synchronous</i>) requests will be made, resulting in <i>low latency</i> and a maximum duration of 1 hour for a single request. For <code>deployment_mode="batch"</code>, batch (<i>asynchronous</i>) requests will be made, resulting in a higher latency but a maximum duration of <i>48 hours</i> for a single request. You will retrieve a request id with which you can obtain the results later (see <i>request retention mode</i>).
<br/><br/>
<b>Request Retention Mode:</b>
Use <code>request_retention_mode="metadata"</code> or <code>request_retention_mode="full"</code> to store all direct and batch requests. <u>Only then you will be able to retrieve them later</u>. In this demo, we don't need to store them.

</div>

In [ ]:
version <- list(
    version = DEPLOYMENT_VERSION,
    language = "r4.0",
    instance_type = "256mb",
    maximum_instances = 1,
    minimum_instances = 0,
    maximum_idle_time = 1800, # = 30 minutes
    deployment_mode = "express",  # 'express' or 'batch'
    request_retention_mode="none"  # We don't need to store the requests in this demo
)

result <- deployment_versions_create(
    deployment.name = DEPLOYMENT_NAME,
    data = version
)
result

Upload the prepared deployment file to the created deployment version. This will create a revision and starts a build.

In [ ]:
result <- revisions_file_upload(
    deployment.name = DEPLOYMENT_NAME,
    version = DEPLOYMENT_VERSION,
    file = deployment_package_zip
)
build.id <- result$build
result

Check if deployment is finished building.
This can take a few minutes.

In [ ]:
status <- "queued"
while (status != "success" && status != "failed") {
    result <- builds_get(
        deployment.name = DEPLOYMENT_NAME,
        version = DEPLOYMENT_VERSION,
        build.id = build.id
    )
    status <- result$status
    Sys.sleep(2)
}
status

The version is now available.

In [ ]:
result <- deployment_versions_get(
    deployment.name = DEPLOYMENT_NAME,
    version = DEPLOYMENT_VERSION
)
result$status

## Creating a request
Make sure the deployment is in 'available' state before performing deployment requests.

The first request is usually slow because of a cold start. A second request (performed within the `maximum_idle_time`, which was set to 30 minutes in this tutorial) will be much faster.

<div class="alert alert-block alert-info">
    
<b>Batch deployments:</b>
For deployments with <code>deployment_mode="batch"</code>, it's not possible to create a <i>direct</i> request. You should create a <i>batch</i> (asynchronous) request instead: use <code>batch_deployment_requests_create</code> instead of <code>deployment_requests_create</code>:<br/>

<pre><code r>data &lt;- list(input = 123)
request_batch &lt;- batch_deployment_requests_create(
    deployment.name = DEPLOYMENT_NAME,
    data = list(data)
)
print(request_batch)
</code></pre>

<br/>Please, note that it is only possible to retrieve the results later if you used <code>request_retention_mode="full"</code> upon deployment version creation.<br/>

<pre><code r>request_result &lt;- deployment_requests_get(
    deployment.name = DEPLOYMENT_NAME,
    request.id = request_batch[[1]][["id"]]
)
print(request_result)
</code></pre>
</div>

In [ ]:
input_data <- list(input = 123)

result <- deployment_version_requests_create(
    deployment.name = DEPLOYMENT_NAME,
    version = DEPLOYMENT_VERSION,
    data = input_data
)
result

It's possible to make a request to the *default* deployment version. The first created version automatically becomes the *default* version, and can be set to another version later. 

In [ ]:
input_data <- list(input = 123)

result <- deployment_requests_create(
    deployment.name = DEPLOYMENT_NAME,
    data = input_data
)
result

***

# Pipelines

Multiple deployments can be chained using a *pipeline*. In this tutorial, we will chain the deployment created above with a new deployment. Just for demo purposes, we will use the same deployment package (the same code) for both deployments in the pipeline. Therefore, the pipeline will contain two modelling steps: each step will multiply the input by a random number.

## Create another deployment and version
Normally you would like to chain different deployments. However, just for demo purposes, we will use the same deployment package as used above.

Create the deployment and version.

In [ ]:
deployment2 <- deployment
deployment2$name <- DEPLOYMENT_NAME2

version2 <- version
version2$version <- DEPLOYMENT_VERSION2

deployments_create(data = deployment2)
deployment_versions_create(deployment.name = DEPLOYMENT_NAME2, data = version2)

Upload the zip.

In [ ]:
result <- revisions_file_upload(
    deployment.name = DEPLOYMENT_NAME2,
    version = DEPLOYMENT_VERSION2,
    file = deployment_package_zip
)
build.id2 <- result$build
result

Wait for the deployment to be ready.

In [ ]:
status <- "queued"
while (status != "success" && status != "failed") {
    result <- builds_get(
        deployment.name = DEPLOYMENT_NAME2,
        version = DEPLOYMENT_VERSION2,
        build.id = build.id2
    )
    status <- result$status
    Sys.sleep(2)
}
status

## Create a pipeline and version

In [ ]:
pipeline <- list(
    name = PIPELINE_NAME,
    description = "A simple pipeline that multiplies the input float two times by a random number.",
    input_type = "structured",
    input_fields = list(
        list(name = "input", data_type = "double")
    ),
    output_type = "structured",
    output_fields = list(
        list(name = "output", data_type = "double")
    ),
    labels = list(demo = "r-quickstart")
)
result <- pipelines_create(data = pipeline)
result

<div class="alert alert-block alert-info">
    
<b>Request Retention Mode:</b>
Use <code>request_retention_mode="metadata"</code> or <code>request_retention_mode="full"</code> to store all direct and batch requests to the pipeline version. Only then you will be able to retrieve them later. In this demo, we don't need to store them.
</div>

In [ ]:
pipeline_version <- list(
    version = PIPELINE_VERSION,
    request_retention_mode='none'  # We don't need to store the requests in this de
)

result <- pipeline_versions_create(
    pipeline.name = PIPELINE_NAME,
    data = pipeline_version
)
result

## Add deployments as objects to pipeline version

In [ ]:
object <- list(
    name = DEPLOYMENT_NAME,
    reference_name = DEPLOYMENT_NAME,
    version = DEPLOYMENT_VERSION
)
result <- pipeline_version_objects_create(
    pipeline.name = PIPELINE_NAME,
    version = PIPELINE_VERSION,
    data = object
)
result

It's also possible to refer to the *default* deployment version. Let's do that for the second object.

In [ ]:
object2 <- list(
    name = DEPLOYMENT_NAME2,
    reference_name = DEPLOYMENT_NAME2
)

result <- pipeline_version_objects_create(
    pipeline.name = PIPELINE_NAME,
    version = PIPELINE_VERSION,
    data = object2
)
result

## Connect the objects

start -> deployment1

In [ ]:
attachment <- list(
    destination_name = DEPLOYMENT_NAME,
    sources = list(
        list(
            source_name = "pipeline_start",
            mapping = list(
                list(source_field_name = "input", destination_field_name = "input")
            )
        )
    )
)
result <- pipeline_version_object_attachments_create(
    pipeline.name = PIPELINE_NAME, 
    version = PIPELINE_VERSION,
    data = attachment
)
result

deployment1 -> deployment2

In [ ]:
attachment2 <- list(
    destination_name = DEPLOYMENT_NAME2,
    sources = list(
        list(
            source_name = DEPLOYMENT_NAME,
            mapping = list(
                list(source_field_name = "output", destination_field_name = "input")
            )
        )
    )
)
result <- pipeline_version_object_attachments_create(
    pipeline.name = PIPELINE_NAME, 
    version = PIPELINE_VERSION,
    data = attachment2
)
result

deployment2 -> end

In [ ]:
attachment <- list(
    destination_name = "pipeline_end",
    sources = list(
        list(
            source_name = DEPLOYMENT_NAME2,
            mapping = list(
                list(source_field_name = "output", destination_field_name = "output")
            )
        )
    )
)
result <- pipeline_version_object_attachments_create(
    pipeline.name = PIPELINE_NAME, 
    version = PIPELINE_VERSION,
    data = attachment
)
result

## Creating a direct request

<div class="alert alert-block alert-info">
    
<b>Batch deployments:</b>
If your pipeline contains one or more deployments with <code>deployment_mode="batch"</code>, it's not possible to create a pipeline <i>direct</i> request. You should create a pipeline <i>batch</i> request instead: use <code>batch_pipeline_requests_create</code> instead of <code>pipeline_requests_create</code>.<br/>

<pre><code python>data &lt;- list(input = 123)
pipeline_batch &lt;- batch_pipeline_requests_create(
    pipeline.name = PIPELINE_NAME,
    data = list(data)
)
print(pipeline_batch)
</code></pre>

<br/>Please, note that it is only possible to retrieve the results later if you use <code>request_retention_mode="full"</code>.<br/>

<pre><code python>pipeline_result &lt;- pipeline_requests_get(
    pipeline.name = PIPELINE_NAME,
    request.id = pipeline_batch[[1]][["id"]]
)
print(pipeline_result)
</code></pre>
</div>

In [ ]:
input_data <- list(input = 123)

result <- pipeline_version_requests_create(
    pipeline.name = PIPELINE_NAME,
    version = PIPELINE_VERSION,
    data = input_data
)
result

It's also possible to make a request to the *default* pipeline version. Like for deployments, the first created pipeline version automatically becomes the *default* version, and can be set to another version later.

In [ ]:
input_data <- list(input = 123)

result <- pipeline_requests_create(
    pipeline.name = PIPELINE_NAME,
    data = input_data
)
result

## Cleanup
Delete created pipeline.

In [ ]:
pipelines_delete(pipeline.name = PIPELINE_NAME)

Delete created deployments.

In [ ]:
deployments_delete(deployment.name = DEPLOYMENT_NAME)
deployments_delete(deployment.name = DEPLOYMENT_NAME2)